In [1]:
from func import DensityMatrix, Operator, QuantumChannel, Ket, Bra
import numpy as np

**Simulating Amplitude Damping**

Evolution of a qubit undergoing amplitude damping(T1 decay)

In [2]:

# density matrix of initial state |1>
rho0 = DensityMatrix([[0,0],[0,1]])

gamma_values = [0, 0.2, 0.5, 0.8, 1.0]
for gamma in gamma_values:
    channel = QuantumChannel.amplitude_damping(gamma)
    rho_t = channel.apply(rho0)
    print(f"Gamma: {gamma}")
    print(rho_t.matrix)
    



Gamma: 0
[[0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
Gamma: 0.2
[[0.2+0.j 0. +0.j]
 [0. +0.j 0.8+0.j]]
Gamma: 0.5
[[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]]
Gamma: 0.8
[[0.8+0.j 0. +0.j]
 [0. +0.j 0.2+0.j]]
Gamma: 1.0
[[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


**Quantum Teleportation**

In [3]:
# Quantum State to be teleported: |ψ> = (|0> + |1>)/√2
psi = (Ket([1,0]) + Ket([0,1])) * (1/np.sqrt(2))
rho_psi = DensityMatrix(psi.outer_product(psi.dagger()))
print("Density Matrix of |ψ>:")
print(rho_psi.matrix)

# Define Bell State |Φ+> = (|00> + |11>)/√2
bell_state = (Ket([1,0,0,0]) + Ket([0,0,0,1])) * (1/np.sqrt(2))
rho_bell = DensityMatrix(bell_state.outer_product(bell_state.dagger()))
print("Density Matrix of Bell State |Φ+>:")
print(rho_bell.matrix)

# Combined initial state: |ψ> ⊗ |Φ+>
psi_combined = psi.tensor(bell_state)
rho_combined = DensityMatrix(psi_combined.outer_product(psi_combined.dagger()))
print("Combined Density Matrix of |ψ> ⊗ |Φ+>:")
print(rho_combined.matrix)

# Kraus operators for amplitude damping channel
gamma = 0.2  # Damping probability
K0 = Operator([[1, 0], [0, np.sqrt(1 - gamma)]])
K1 = Operator([[0, np.sqrt(gamma)], [0, 0]])

# Making kraus operator for 3 qubits to apply on last two qubits
M0 = Operator(Operator.identity).tensor(K0, K0)
M1 = Operator(Operator.identity).tensor(K0, K1)
M2 = Operator(Operator.identity).tensor(K1, K0)
M3 = Operator(Operator.identity).tensor(K1, K1)
krauss_3qubit = [M0, M1, M2, M3]

# Applying amplitude damping channel on last two qubits
channel_3qubit = QuantumChannel(krauss_3qubit)
rho_damped = channel_3qubit.apply(rho_combined)

# Applying CNot on first two qubit and Hadamard on first Qubit
CNot = Operator.cnot(0, 1, 3)  
Hadamard = Operator.hadamard(0, 3)  
state_1 = rho_damped.evolve(CNot)
state_2 = state_1.evolve(Hadamard)

#print('State after teleportation operations:')
#print(state_2.matrix)

# Measurement on Qubit 0 and 1 (first two qubits)
P0 = Operator([[1, 0], [0, 0]])
P1 = Operator([[0, 0], [0, 1]])

# Measurement operators for qubits 0 and 1
M00 = P0.tensor(P0, Operator(Operator.identity))
M01 = P0.tensor(P1, Operator(Operator.identity))
M10 = P1.tensor(P0, Operator(Operator.identity))
M11 = P1.tensor(P1, Operator(Operator.identity))

# Get measurement probabilities
prob_00 = np.real_if_close(np.trace(M00.matrix @ state_2.matrix))
prob_01 = np.real_if_close(np.trace(M01.matrix @ state_2.matrix))
prob_10 = np.real_if_close(np.trace(M10.matrix @ state_2.matrix))
prob_11 = np.real_if_close(np.trace(M11.matrix @ state_2.matrix))

print("Measurement probabilities for qubits 0 and 1:")
print(f"P(|00⟩) = {prob_00:.4f}")
print(f"P(|01⟩) = {prob_01:.4f}")
print(f"P(|10⟩) = {prob_10:.4f}")
print(f"P(|11⟩) = {prob_11:.4f}")

# Apply corrections on third qubit based on measurement results
X_gate = Operator(Operator.pauli_x)
Z_gate = Operator(Operator.pauli_z)

original_psi = (Ket([1,0]) + Ket([0,1])) * (1/np.sqrt(2))
rho_target = DensityMatrix(original_psi.outer_product(original_psi.dagger()))

fidelities = []

for outcome, prob, correction in [("00", prob_00, Operator(Operator.identity)),
                                  ("01", prob_01, X_gate),
                                  ("10", prob_10, Z_gate),
                                  ("11", prob_11, X_gate @ Z_gate)]:
    
    if prob > 1e-10:
        # Get post-measurement state
        M_outcome = locals()[f"M{outcome}"]
        rho_post = DensityMatrix((M_outcome.matrix @ state_2.matrix @ M_outcome.matrix) / prob)
        
        # Get third qubit state (the teleported qubit) - FIXED: keep qubit 2
        rho_third = rho_post.partial_trace(keep=[2], dims=[2, 2, 2])
        
        # Apply correction
        rho_corrected = rho_third.evolve(correction)
        
        # Calculate fidelity with original state
        fidelity = rho_corrected.fidelity(rho_target)
        fidelities.append(fidelity)
        
        print(f"\nOutcome {outcome}:")
        print(f"Third qubit after correction:\n{rho_corrected.matrix}")
        print(f"Fidelity with original |ψ⟩: {fidelity:.6f}")

# Calculate average fidelity
if fidelities:
    avg_fidelity = sum(f * p for f, p in zip(fidelities, [prob_00, prob_01, prob_10, prob_11]))
    print(f"\nAverage fidelity: {avg_fidelity:.6f}")

Density Matrix of |ψ>:
[[0.5 0.5]
 [0.5 0.5]]
Density Matrix of Bell State |Φ+>:
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]
Combined Density Matrix of |ψ> ⊗ |Φ+>:
[[0.25 0.   0.   0.25 0.25 0.   0.   0.25]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.25 0.   0.   0.25 0.25 0.   0.   0.25]
 [0.25 0.   0.   0.25 0.25 0.   0.   0.25]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.25 0.   0.   0.25 0.25 0.   0.   0.25]]
Measurement probabilities for qubits 0 and 1:
P(|00⟩) = 0.8400
P(|01⟩) = 0.1600
P(|10⟩) = 0.0000
P(|11⟩) = -0.0000

Outcome 00:
Third qubit after correction:
[[0.61904762+0.j 0.47619048+0.j]
 [0.47619048+0.j 0.38095238+0.j]]
Fidelity with original |ψ⟩: 0.976190

Outcome 01:
Third qubit after correction:
[[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]]
Fidelity with original |ψ⟩: 0.500000

Average fidelity: 0.900000


/workspaces/qc_lib/func.py:338: LinAlgWarning: Matrix is singular. The result might be inaccurate or the array might not have a square root.
  sqrt_product = scipy.linalg.sqrtm(product)


### Quantum Teleportation from General Code in Library

In [ ]:

state = (Ket([1,0]) + Ket([0,1]) )* (1/np.sqrt(2))
result = QuantumChannel.quantum_teleportation(state, gamma=0.2)
print(result)


=== QUANTUM TELEPORTATION PROTOCOL ===
Initial state: [0.70710678 0.70710678]
Noise parameter gamma: 0.2

Measurement probabilities:
P(|00⟩) = 0.8400
P(|01⟩) = 0.1600
P(|10⟩) = 0.0000
P(|11⟩) = 0.0000

Outcome 00:
Correction applied: [[1 0]
 [0 1]]
Teleported state:
[[0.61904762+0.j 0.47619048+0.j]
 [0.47619048+0.j 0.38095238+0.j]]
Fidelity with original: 0.976190

Outcome 01:
Correction applied: [[0 1]
 [1 0]]
Teleported state:
[[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]]
Fidelity with original: 0.500000

=== RESULTS SUMMARY ===
Average fidelity: 0.900000
{'initial_state': Ket([0.70710678 0.70710678]), 'gamma': 0.2, 'measurement_probabilities': {'00': np.float64(0.84), '01': np.float64(0.16), '10': np.float64(1.356219139056376e-35), '11': np.float64(0.0)}, 'fidelities': {'00': np.float64(0.9761904936451891), '01': np.float64(0.4999999999999991)}, 'average_fidelity': np.float64(0.9000000146619587), 'post_measurement_states': {'00': Operator([[0.61904762+0.j 0.47619048+0.j]
 [0.47619048+0.j 0

In [10]:
print("Example 1: |+⟩ state with no noise")
result1 = QuantumChannel.quantum_teleportation(
    initial_state=(Ket([1,0]) + Ket([0,1])) * (1/np.sqrt(2)),
    gamma=0.0
)

print("\n" + "="*50)
print("Example 2: |0⟩ state with moderate noise")
result2 = QuantumChannel.quantum_teleportation(
    initial_state=Ket([1,0]),
    gamma=0.3
)

print("\n" + "="*50)
print("Example 3: Custom superposition with high noise")
result3 = QuantumChannel.quantum_teleportation(
    initial_state=(Ket([1,0]) * 0.6 + Ket([0,1]) * 0.8),  # Will be normalized
    gamma=0.7
)



Example 1: |+⟩ state with no noise
=== QUANTUM TELEPORTATION PROTOCOL ===
Initial state: [0.70710678 0.70710678]
Noise parameter gamma: 0.0

Measurement probabilities:
P(|00⟩) = 1.0000
P(|01⟩) = 0.0000
P(|10⟩) = 0.0000
P(|11⟩) = 0.0000

Outcome 00:
Correction applied: [[1 0]
 [0 1]]
Teleported state:
[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]
Fidelity with original: 1.000000-0.000000j

=== RESULTS SUMMARY ===
Average fidelity: 1.000000-0.000000j

Example 2: |0⟩ state with moderate noise
=== QUANTUM TELEPORTATION PROTOCOL ===
Initial state: [1. 0.]
Noise parameter gamma: 0.3

Measurement probabilities:
P(|00⟩) = 0.3950
P(|01⟩) = 0.1050
P(|10⟩) = 0.3950
P(|11⟩) = 0.1050

Outcome 00:
Correction applied: [[1 0]
 [0 1]]
Teleported state:
[[0.68987342+0.j 0.44303797+0.j]
 [0.44303797+0.j 0.31012658+0.j]]
Fidelity with original: 0.689873

Outcome 01:
Correction applied: [[0 1]
 [1 0]]
Teleported state:
[[0.5+0.j 0. +0.j]
 [0. +0.j 0.5+0.j]]
Fidelity with original: 0.500000

Outcome 10:
Correction